In [16]:
!pip install bokeh


In [17]:
import pandas as pd
import numpy as np
import time
from bokeh.plotting import figure, show, output_notebook
output_notebook()

In [18]:
# Model settings
BASE_PRICE = 10

def linear_price_model(prev_price, occupancy, capacity, alpha=5):
    utilization = occupancy / capacity
    return prev_price + alpha * utilization

def demand_function(occupancy, capacity, queue, traffic, special_day, vehicle_type):
    vt_weight = {'car': 1, 'bike': 0.5, 'truck': 1.5}
    return (
        1.0 * (occupancy / capacity) +
        0.8 * queue -
        0.6 * traffic +
        0.5 * special_day +
        0.7 * vt_weight.get(vehicle_type.lower(), 1.0)
    )


In [19]:
def bounded_price(base, demand, lam=0.1):
    norm_demand = (demand - 2) / 5
    price = base * (1 + lam * norm_demand)
    return np.clip(price, base * 0.5, base * 2)

def competitive_price_model(own_price, utilization, competitors):
    cheaper = [c for c in competitors if c < own_price]
    if utilization > 0.9 and cheaper:
        return own_price * 0.9
    elif all(c > own_price for c in competitors):
        return own_price * 1.1
    return own_price

In [22]:
# Load dataset
df = pd.read_csv("/content/dataset.csv")

# Store output
price_log = []

# Process row by row (simulate streaming)
prev_price = BASE_PRICE

In [24]:
df.columns

Index(['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude',
       'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength',
       'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime'],
      dtype='object')

In [33]:
df["Occupancy"] = pd.to_numeric(df["Occupancy"], errors="coerce")
df["Capacity"] = pd.to_numeric(df["Capacity"], errors="coerce")
df["QueueLength"] = pd.to_numeric(df["QueueLength"], errors="coerce")
df["TrafficConditionNearby"] = pd.to_numeric(df["TrafficConditionNearby"], errors="coerce")
df["IsSpecialDay"] = pd.to_numeric(df["IsSpecialDay"], errors="coerce")
df["timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"], dayfirst=True)



for i, row in df.iterrows():
    # Model 1: linear baseline
    price1 = linear_price_model(prev_price, row["Occupancy"], row["Capacity"])

    # Model 2: demand-based
    demand = demand_function(row["Occupancy"], row["Capacity"], row["QueueLength"],
                             row["TrafficConditionNearby"], row["IsSpecialDay"], row["VehicleType"])
    price2 = bounded_price(BASE_PRICE, demand)

    # Model 3: competitive (simulate 3 random competitor prices)
    competitors = [BASE_PRICE + np.random.uniform(-2, 2) for _ in range(3)]
    final_price = competitive_price_model(price2, row["Occupancy"] / row["Capacity"], competitors)

    # Save the price
    price_log.append({
        "timestamp": row["timestamp"],
        "lot_id": row["ID"],
        "final_price": round(final_price, 2)
    })

    # Update previous price
    prev_price = final_price

    # Optional delay for realism
    time.sleep(0.01)  # simulate 10 ms delay